In [1]:
import os
import pandas as pd
import matplotlib as plt
import gpboost as gpb
import sklearn

In [2]:
treatment = pd.read_csv("treatments_2017-2020.csv")

In [3]:
# filter out the treatment completed one, i.e.REASON == 1
case1 = treatment[treatment['REASON'] == 1] 

In [4]:
# sum of days for each substances, need to be averaged later by total count
sub1_lossum = case1.groupby(['SUB1','REGION'])['LOS'].sum().to_frame()
sub1_lossum.rename(columns={'LOS':'LOS_sum'}, inplace=True)
sub1_lossum.reset_index(inplace=True)
sub1_lossum

,SUB1,REGION,LOS_sum
0,-9,0,304
1,-9,1,15623
2,-9,2,18878
3,-9,3,52575
4,-9,4,177902
...,...,...,...
88,19,0,423
89,19,1,47100
90,19,2,10074
91,19,3,44211


In [5]:
sub1_cases = treatment.groupby(['SUB1', 'REGION'])['REASON'].sum().to_frame()
sub1_cases.rename(columns={'REASON':'count_sum'}, inplace=True)
sub1_cases.reset_index(inplace=True)
sub1_cases

,SUB1,REGION,count_sum
0,-9,0,36
1,-9,1,3578
2,-9,2,7120
3,-9,3,14954
4,-9,4,44531
...,...,...,...
90,19,0,72
91,19,1,11105
92,19,2,3064
93,19,3,36931


In [6]:
sub1_ct_lossum = pd.merge(sub1_cases, sub1_lossum)
sub1_ct_lossum

,SUB1,REGION,count_sum,LOS_sum
0,-9,0,36,304
1,-9,1,3578,15623
2,-9,2,7120,18878
3,-9,3,14954,52575
4,-9,4,44531,177902
...,...,...,...,...
88,19,0,72,423
89,19,1,11105,47100
90,19,2,3064,10074
91,19,3,36931,44211


In [7]:
sub1_ct_lossum['los_average'] = sub1_ct_lossum['LOS_sum']/sub1_ct_lossum['count_sum']
sub1_ct_lossum

,SUB1,REGION,count_sum,LOS_sum,los_average
0,-9,0,36,304,8.444444
1,-9,1,3578,15623,4.366406
2,-9,2,7120,18878,2.651404
3,-9,3,14954,52575,3.515782
4,-9,4,44531,177902,3.995015
...,...,...,...,...,...
88,19,0,72,423,5.875000
89,19,1,11105,47100,4.241333
90,19,2,3064,10074,3.287859
91,19,3,36931,44211,1.197124


In [8]:
case1 = treatment[treatment['REASON'] == 1]
sub1_case1 = case1.groupby(['SUB1','REGION']).count()
sub1_case1.reset_index(inplace=True)
sub1_case1.rename(columns={'REASON':'count_case1'}, inplace=True)
sub1_case1_filt = sub1_case1[['SUB1','REGION', 'count_case1']]
sub1_case1_filt

,SUB1,REGION,count_case1
0,-9,0,10
1,-9,1,557
2,-9,2,1022
3,-9,3,1684
4,-9,4,16353
...,...,...,...
88,19,0,13
89,19,1,2210
90,19,2,389
91,19,3,1862


In [9]:
sub1_cal = pd.merge(sub1_cases, sub1_case1_filt)
sub1_cal['comp_rate'] = sub1_cal['count_case1']/sub1_cal['count_sum']
sub1_cal

,SUB1,REGION,count_sum,count_case1,comp_rate
0,-9,0,36,10,0.277778
1,-9,1,3578,557,0.155674
2,-9,2,7120,1022,0.143539
3,-9,3,14954,1684,0.112612
4,-9,4,44531,16353,0.367227
...,...,...,...,...,...
88,19,0,72,13,0.180556
89,19,1,11105,2210,0.199009
90,19,2,3064,389,0.126958
91,19,3,36931,1862,0.050418


In [10]:
sub1_losaverage_comp_rate_combine = pd.merge(sub1_ct_lossum, sub1_cal)
sub1_losaverage_comp_rate = sub1_losaverage_comp_rate_combine[['SUB1', 'REGION', 'los_average', 'comp_rate']]
sub1_losaverage_comp_rate

,SUB1,REGION,los_average,comp_rate
0,-9,0,8.444444,0.277778
1,-9,1,4.366406,0.155674
2,-9,2,2.651404,0.143539
3,-9,3,3.515782,0.112612
4,-9,4,3.995015,0.367227
...,...,...,...,...
88,19,0,5.875000,0.180556
89,19,1,4.241333,0.199009
90,19,2,3.287859,0.126958
91,19,3,1.197124,0.050418
